In [ ]:
# | default_exp classes.DomoPage


In [ ]:
# |export
from fastcore.basics import patch_to

In [ ]:
# | exporti
from dataclasses import dataclass, field

# import asyncio
# import httpx

import domolibrary.client.DomoAuth as dmda
import domolibrary.classes.DomoUser as dmdu
import domolibrary.utils.DictDot as util_dd

# from .routes import page_routes
# from ..utils.Base import Base

In [ ]:
# | export
@dataclass
class DomoPage:
    id: str
    title: str = None
    parent_page_id: str = None
    auth: dmda.DomoAuth = field(default = None , repr = False)
    owners: list = field(default_factory=list)
    cards: list = field(default_factory=list)
    collections: list = field(default_factory=list)
    children: list = field(default_factory=list)

    def display_url(self):
        return f"https://{self.auth.domo_instance}.domo.com/page/{self.id}"

In [ ]:
# | export


@patch_to(DomoPage, cls_method=True)
def _from_bootstrap(cls: DomoPage, page_obj, auth: dmda.DomoAuth = None):

    dd = page_obj
    if isinstance(page_obj, dict):
        dd = util_dd.DictDot(page_obj)

    domo_page = cls(id=dd.id, title=dd.title, auth=auth)

    if isinstance(dd.owners, list) and len(dd.owners) > 0:
        domo_page.owners = [
            dmdu.DomoUser._from_bootstrap_json(auth=auth, user_obj=user_dd)
            for user_dd in dd.owners
            if user_dd.type == 'USER'
        ]

        [print(other_dd) for other_dd in dd.owners
         if other_dd.type != 'USER']

    if isinstance(dd.children, list) and len(dd.children) > 0:
        domo_page.children = [
            cls._from_bootstrap(page_obj=child_dd, auth=auth)
            for child_dd in dd.children
            if child_dd.type == "page"
        ]

        [print(other_dd) for other_dd in dd.children
            if other_dd.type != "page"]

    return domo_page


In [ ]:
import os
import domolibrary.routes.bootstrap as bootstrap_routes

auth = dmda.DomoFullAuth(
    domo_instance='domo-community', domo_password = os.environ['DOJO_PASSWORD'], domo_username = 'jae@onyxreporting.com')

res = await bootstrap_routes.get_bootstrap_pages(auth = auth)

page_obj = res.response[10]
DomoPage._from_bootstrap(page_obj, auth = auth)

DomoPage(id='790951325', title='Coronavirus (COVID-19) Dashboard', parent_page_id=None, owners=[DomoUser(id=1345408759, display_name='Alexis Lorenz (DataMaven)', email_address=None, role_id=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None)], cards=[], collections=[], children=[DomoPage(id='1289295627', title='Dataset Schemas', parent_page_id=None, owners=[DomoUser(id=1345408759, display_name='Alexis Lorenz (DataMaven)', email_address=None, role_id=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None)], cards=[], collections=[], children=[]), DomoPage(id='756869253', title='Coronavirus (Alexis)', parent_page_id=None, owners=[DomoUser(id=1345408759, display_name='Alexis Lorenz (DataMaven)', email_address=None, role_id=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None)], cards=[], collections=[], children=[]), DomoPage(id='11555470', title='Contiguous United States Detail (Alexis)', parent_page_id=None, owners=[DomoUser

In [ ]:
#| export
@patch_to(DomoPage)
def _ol_fn():
    pass


In [ ]:
# @classmethod
#     async def get_from_id(cls, page_id: str, full_auth: DomoFullAuth, debug: bool = False):
#         import Library.DomoClasses.DomoCard as dc
#         res = await page_routes.get_page_by_id(full_auth=full_auth, page_id=page_id, debug=debug)

#         if res.status == 200:
#             dd = DictDot(res.response)

#             pg = cls(
#                 id=dd.id,
#                 domo_instance=full_auth.domo_instance,
#                 title=dd.title,
#                 parent_page_id=dd.page.parentPageId,
#                 owners=dd.page.owners,
#                 collections=dd.collections
#             )

#             pg.cards = await asyncio.gather(
#                 *[dc.DomoCard.get_from_id(id=card.id, full_auth=full_auth) for card in dd.cards])

#             return pg

#     @classmethod
#     async def get_cards(cls, full_auth, page_id, debug: bool = False, session: httpx.AsyncClient = None):
#         try:
#             import Library.DomoClasses.DomoCard as dc
#             close_session = False if session else True

#             if not session:
#                 session = httpx.AsyncClient()

#             res = await page_routes.get_page_definition(full_auth=full_auth, page_id=page_id, debug=debug, session=session)

#             if res.status == 200:
#                 json = res.response

#                 card_list = [dc.DomoCard(id=card.get(
#                     'id'), full_auth=full_auth) for card in json.get('cards')]

#                 return card_list

#             else:
#                 return None

#         finally:
#             if close_session:
#                 await session.aclose()

#     async def get_datasets(full_auth, page_id, debug: bool = False, session: httpx.AsyncClient = None):
#         try:
#             import Library.DomoClasses.DomoDataset as dmds
#             close_session = False if session else True

#             if not session:
#                 session = httpx.AsyncClient()

#             res = await page_routes.get_page_definition(full_auth=full_auth, page_id=page_id, debug=debug, session=session)

#             if res.status == 200:
#                 json = res.response

#                 dataset_ls = [card.get('datasources')
#                               for card in json.get('cards')]

#                 return [dmds.DomoDataset(id=ds.get('dataSourceId'), full_auth=full_auth) for ds_ls in dataset_ls for ds in ds_ls]

#             else:
#                 return None

#         finally:
#             if close_session:
#                 await session.aclose()

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()